Setup

In [1]:
import time
import argparse
import json
import csv

from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap, PassManager
from pathlib import Path
from typing import Dict, List, Iterable
from util import EAGLE_COUPLING, sabre, count_swaps 
from multilevel_sabre import MultiLevelSabre
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

Coupling map

In [2]:
from util import EAGLE_COUPLING

coupling_map = CouplingMap(couplinglist=EAGLE_COUPLING)
coupling_map.make_symmetric()
#coupling_map.draw()

Select circuits

In [3]:
directory = "circuits_no_swaps/"
result_path = Path("results/comparison_no_swaps.csv")
circuit_files = []
for file in Path(directory).glob("*.qasm"):
    circuit_files.append(file)
print(f"Found {len(circuit_files)} circuit files.")    

Found 7 circuit files.


Setup

In [4]:
random_seed = 1

In [5]:
qasm_file = circuit_files[2]
qc = QuantumCircuit.from_qasm_file(qasm_file)

In [6]:
pm = generate_preset_pass_manager(optimization_level=3, coupling_map=coupling_map, seed_transpiler=random_seed)

In [7]:
qc_tr = pm.run(qc)

In [8]:
def count_swaps(circuit):
    """Count the number of SWAP gates in a circuit.
    
    Args:
        circuit (QuantumCircuit): The circuit to count SWAPs in
        
    Returns:
        int: Number of SWAP gates in the circuit
    """
    return sum(1 for instruction in circuit.data if instruction.operation.name == 'swap')

In [10]:
swaps_count = count_swaps(qc_tr)
depth       = qc_tr.depth()
depth_2q    = qc_tr.depth(lambda x: x.operation.num_qubits == 2)

print(f"Qasm File: {qasm_file}")
print(f"SWAPs: {swaps_count}, Depth: {depth}, 2Q Depth: {depth_2q}")


Qasm File: circuits_no_swaps/dnn_n51.qasm
SWAPs: 6, Depth: 70, 2Q Depth: 65
